In [1]:
import base64
import hmac
import json
import time
import requests
import toml

from urllib.parse import urljoin

# params
base_url = 'https://www.iihvvs.com'
api_config = toml.load('../config/api.toml')
api_key = api_config['api_key']
secret_key = api_config['secret_key']
passphrase = api_config['passphrase']

In [2]:
def request(method, uri, params=None, body=None, headers=None, auth=False):
    """Initiate network request
   @param method: request method, GET / POST / DELETE / PUT
   @param uri: request uri
   @param params: dict, request query params
   @param body: dict, request body
   @param headers: request http header
   @param auth: boolean, add permission verification or not
   """
    if params:
        query = "&".join(
            ["{}={}".format(k, params[k]) for k in sorted(params.keys())]
        )
        uri += "?" + query
    url = urljoin(base_url, uri)

    if auth:
        timestamp = (
                str(time.time()).split(".")[0]
                + "."
                + str(time.time()).split(".")[1][:3]
        )
        if body:
            body = json.dumps(body)
        else:
            body = ""
        message = str(timestamp) + str.upper(method) + uri + str(body)
        mac = hmac.new(
            bytes(secret_key, encoding="utf8"),
            bytes(message, encoding="utf-8"),
            digestmod="sha256",
        )
        d = mac.digest()
        sign = base64.b64encode(d)

        if not headers:
            headers = {}
        headers["Content-Type"] = "application/json"
        headers["OK-ACCESS-KEY"] = api_key
        headers["OK-ACCESS-SIGN"] = sign
        headers["OK-ACCESS-TIMESTAMP"] = str(timestamp)
        headers["OK-ACCESS-PASSPHRASE"] = passphrase
    result = requests.request(
        method, url, data=body, headers=headers, timeout=10
    ).json()
    if result.get("code") and result.get("code") != "0":
        return None, result
    return result, None

In [3]:
def get_asset(currency):
    params = {"ccy": currency}
    result = request(
        "GET", "/api/v5/account/balance", params=params, auth=True
    )
    return result


get_asset('USDT')

({'code': '0',
  'data': [{'adjEq': '',
    'borrowFroz': '',
    'details': [{'availBal': '0.0089344887289903',
      'availEq': '0.0089344887289903',
      'borrowFroz': '',
      'cashBal': '0.0089344887289903',
      'ccy': 'USDT',
      'crossLiab': '',
      'disEq': '251.4651219879153',
      'eq': '251.42740787673375',
      'eqUsd': '251.4651219879153',
      'fixedBal': '0',
      'frozenBal': '251.41847338800477',
      'imr': '0',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '0',
      'liab': '',
      'maxLoan': '',
      'mgnRatio': '',
      'mmr': '0',
      'notionalLever': '0',
      'ordFrozen': '0',
      'rewardBal': '0',
      'smtSyncEq': '0',
      'spotInUseAmt': '',
      'spotIsoBal': '0',
      'stgyEq': '251.41847338800477',
      'twap': '0',
      'uTime': '1710390749220',
      'upl': '0',
      'uplLiab': ''}],
    'imr': '',
    'isoEq': '0',
    'mgnRatio': '',
    'mmr': '',
    'notionalUsd': '',
    'ordFroz': '',

In [9]:
def get_grid_order_list(order_type):
    params = {"algoOrdType": order_type}
    result = request(
        "GET", "/api/v5/tradingBot/grid/orders-algo-pending", params=params, auth=True
    )
    return result


# get spot grid
result = get_grid_order_list(order_type='grid')
result

({'code': '0',
  'data': [{'actualLever': '',
    'algoClOrdId': '',
    'algoId': '688176870327844864',
    'algoOrdType': 'grid',
    'arbitrageNum': '7',
    'availEq': '',
    'basePos': False,
    'baseSz': '0',
    'cTime': '1710346151601',
    'cancelType': '0',
    'copyType': '0',
    'direction': '',
    'floatProfit': '0.627046753234',
    'gridNum': '20',
    'gridProfit': '0.4357140544',
    'instFamily': '',
    'instId': 'SOL-USDT',
    'instType': 'SPOT',
    'investment': '38',
    'lever': '',
    'liqPx': '',
    'maxPx': '180',
    'minPx': '140',
    'ordFrozen': '',
    'pnlRatio': '0.0279673896745789',
    'profitSharingRatio': '',
    'quoteSz': '38',
    'rebateTrans': [{'rebate': '0', 'rebateCcy': 'SOL'},
     {'rebate': '0', 'rebateCcy': 'USDT'}],
    'runType': '1',
    'slTriggerPx': '',
    'state': 'running',
    'stopType': '',
    'sz': '',
    'tag': '',
    'totalPnl': '1.062760807634',
    'tpTriggerPx': '',
    'triggerParams': [{'triggerAction': 's

In [5]:
# get future grid
result = get_grid_order_list(order_type='contract_grid')
result[0]['data']

[{'actualLever': '1.7930086823313554',
  'algoClOrdId': '',
  'algoId': '688097705075412992',
  'algoOrdType': 'contract_grid',
  'arbitrageNum': '19',
  'availEq': '',
  'basePos': True,
  'baseSz': '',
  'cTime': '1710327277131',
  'cancelType': '0',
  'copyType': '0',
  'direction': 'long',
  'floatProfit': '-1.330893202865814',
  'gridNum': '20',
  'gridProfit': '2.684848',
  'instFamily': 'MATIC-USDT',
  'instId': 'MATIC-USDT-SWAP',
  'instType': 'SWAP',
  'investment': '40',
  'lever': '10',
  'liqPx': '0.5504203625858646',
  'maxPx': '1.3',
  'minPx': '1.1',
  'ordFrozen': '',
  'pnlRatio': '0.0338488699283547',
  'profitSharingRatio': '',
  'quoteSz': '',
  'rebateTrans': [{'rebate': '0', 'rebateCcy': 'USDT'}],
  'runType': '1',
  'slRatio': '',
  'slTriggerPx': '',
  'state': 'running',
  'stopType': '1',
  'sz': '40',
  'tag': '',
  'totalPnl': '1.353954797134186',
  'tpRatio': '',
  'tpTriggerPx': '',
  'triggerParams': [{'triggerAction': 'start',
    'delaySeconds': '0',
  

In [6]:
def get_grid_order_details(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }
    result = request("GET", "/api/v5/tradingBot/grid/orders-algo-details", params=params, auth=True)
    return result


result = get_grid_order_details('grid', 687405759617044480)
result

({'code': '0',
  'data': [{'activeOrdNum': '25',
    'actualLever': '',
    'algoClOrdId': '',
    'algoId': '687405759617044480',
    'algoOrdType': 'grid',
    'annualizedRate': '0.9770399331823109',
    'arbitrageNum': '92',
    'availEq': '',
    'basePos': False,
    'baseSz': '0',
    'cTime': '1710162304481',
    'cancelType': '0',
    'copyType': '0',
    'curBaseSz': '0.00010556056',
    'curQuoteSz': '16.06273151448',
    'direction': '',
    'eq': '',
    'floatProfit': '0.114081166304',
    'gridNum': '25',
    'gridProfit': '0.16484537448',
    'instFamily': '',
    'instId': 'BTC-USDT',
    'instType': 'SPOT',
    'investment': '23.46',
    'lever': '',
    'liqPx': '',
    'maxPx': '75000',
    'minPx': '68500',
    'ordFrozen': '',
    'perMaxProfitRate': '0.0017890329927007255562',
    'perMinProfitRate': '0.0014727722772276442034',
    'pnlRatio': '0.0118894518663256',
    'profit': '0.16484537',
    'profitSharingRatio': '',
    'quoteSz': '23.46',
    'rebateTrans':

In [12]:
def get_grid_sub_orders(order_type, algo_id, order_status):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
        "type": order_status,
    }

    result = request("GET", "/api/v5/tradingBot/grid/sub-orders", params=params, auth=True)
    return result

result = get_grid_sub_orders('contract_grid', '688176870327844864', "live")
len(result[0]['data'])
result[0]['data'][0]

TypeError: 'NoneType' object is not subscriptable

In [8]:
def get_grid_positions(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }

    result = request("GET", "/api/v5/tradingBot/grid/positions", params=params, auth=True)
    return result

result = get_grid_positions('contract_grid', '687008144513568768')
result

(None,
 {'code': '51291',
  'data': [],
  'msg': 'The bot doesn’t exist or has already stopped'})